In [28]:
import pandas as ps
import numpy as nm
import math as mt
import copy

class Node :
    def __init__( self, attribute_name, data, attribute_values, select_from, numeric, entropy ) :
        self.attribute_name = attribute_name
        self.data = data 
        self.attribute_values = attribute_values
        self.childrens = []
        self.select_from = select_from
        self.numeric = numeric
        self.entropy = entropy
    def add_child( self, child_obj ):
        self.childrens.append( child_obj )   

def value_entropy_numerical( lable, att, val, data_u ):
    
    att_col = ps.Series( data_u[ att ] ).tolist()#fecthing out attribute's column as a list so that its access could be easier
    lable_col = ps.Series( data_u[ lable ] ).tolist()#Fetching out whole attribute of lable for same reason as above 
    
    count1 = 0
    count = 0 
    i = 0
    proT = 0
    proF = 0
    
    while( i < len( lable_col ) ):#Counting size and number of ones for that(val) value of this attribute
        if( att_col[ i ] == val ):
            count = count + 1
            if( lable_col[ i ] == 1 ) :
                count1 = count1 + 1 #for numbmer of ones
        i = i + 1         

    proT = count1 / count
    proF = 1 - proT
    if( count1 == count or count1 == 0):
        entropy = 0
    else :    
        entropy = ( proT * mt.log( proT )/mt.log( 2 ) + proF * mt.log( proF )/mt.log( 2 ) ) * ( -1 )
    
    return entropy          
        
        
def value_entropy( lable, att, val, data_u ):
#     att_col = ps.Series( data_u[ att ] ).tolist()
    lable_col = ps.Series( data_u[ lable ] ).tolist()#fecthing out lable attribute's column as a list so that its access could be easier 
    count1 = 0
    count = len( lable_col )
    i = 0
    proT = 0
    proF = 0
    while( i < len( lable_col ) ):#Counting size and number of ones for that(val) value of this attribute
        if( lable_col[ i ] == 1 ) :
            count1 = count1 + 1 #for numbmer of ones
        i = i + 1         

    proT = count1 / count
    proF = 1 - proT
    if( count1 == count or count1 == 0):
        entropy = 0
    else :    
        entropy = ( proT * mt.log( proT )/mt.log( 2 ) + proF * mt.log( proF )/mt.log( 2 ) ) * ( -1 )
    
    return entropy        

def calc_entropy( curr_data, lable ) :
    lable_col = ps.Series( curr_data[ lable ] )
    count1 = 0
    count = 0
    pro1 = 0 
    pro0 = 0
    
    for val in lable_col :
        if( val == 1 ) :
            count1 = count1 + 1
        count = count + 1
        
    pro1 = count1 / count
    pro0 = 1 - pro1
    if( count1 == count or count1 == 0 ) :
        entropy = 0
    else :    
        entropy = ( pro1 * ( mt.log( pro1 ) / mt.log( 2 ) ) + pro0 * ( mt.log( pro0 ) / mt.log( 2 ) ) ) * ( -1 )
    
    return entropy

def select_attribute( curr_data, select_from, numerical_attribute, parent_entropy ):
    
    p_row, p_col = curr_data.shape
    ff = 0
    gain_ratio = -10
    vals_att = []
    initialized = 0
#     print("\n\nSelection of attribute")
    
    
    for attribute in select_from :
#         print("\nChecking attribute ", attribute )
        
        att_size = p_row #It will be equal to dataset available for parent 
        val_size = []#to store size of class after split using any particular value of that attribute
        val_ent = []#to store entropy of class after split using any particular value of that attribute
        
        if( attribute not in numerical_attribute ) :
            vals_att = curr_data[ attribute ].unique().tolist()#getting all unique values of a particular col
#             print("1.Not numerical attribute with values ", vals_att )
            if( len( vals_att ) == 1 ):
                continue
            vals_attribute = ps.Series( vals_att ) 
            
            for val in vals_attribute  :
                c_row, c_col = curr_data[ curr_data[ attribute ] == val ].shape
                val_size.append( c_row )
                val_ent.append( value_entropy( lable, attribute, val, curr_data[ curr_data[ attribute ] == val ] ) )#Calling for entropy
        else :
            del vals_att[:]
#             print("Numerical attribute")
            curr_data.sort_values( attribute )
            attribute_col = curr_data[ attribute ].unique().tolist()#fetch out whole column of particular attribute as a list,
            #So that we can easily access it using indexing, in filling vals_attribute
            if( len( attribute_col ) == 1 ) :
                continue
                
            attribute_col.sort()
            
            if( len( attribute_col ) > 5 ): #size of fecthed out list is more than 5, so that we can divide whole data set into 5 parts
                #Equal to 5 is not considered because if it is equal to five then we will split it in indivisual branches
                div = mt.floor( len( attribute_col ) / 5 )
                insert = 0
                index = div
#                 print("Type of value in dataframe ", type( curr_data[ attribute ] ), "and type of array having unique values ", type( vals_attribute ) )
                while( insert < 4 ) :
                    if attribute_col[ index ] not in vals_att :
                        vals_att.append( attribute_col[ index ] )
                    index = div + index
                    insert = insert + 1
                if attribute_col[ -1 ] not in vals_att :    
                    vals_att.append( attribute_col[ -1 ] )    
            else : 
                insert = 0
                while( insert < len( attribute_col ) ) :#To be removed, 
#                     if attribute_col[ insert ] not in vals_att : #Commented because now we are taking unique in starting only, which eliminates this conditions possibility
                    vals_att.append( attribute_col[ insert ] )
                    insert = insert + 1
            vals_attribute = ps.Series( vals_att )    
#             print("2. Vals attribute updated ", vals_attribute, " type ", type( vals_attribute ) )
            index = 0
            while( index < len( vals_attribute ) ) :
#                 print("Type of value in dataframe ", type( curr_data[ attribute ] ), " and value is ", vals_attribute[ index ] )
                c_row, c_col = curr_data[ curr_data[ attribute ] <= vals_attribute[ index ] ].shape
                val_size.append( c_row )
                val_ent.append( value_entropy_numerical( lable, attribute, vals_attribute[ index ], curr_data ) )
                index = index + 1
                
        info = 0
        i = 0
        while( i < len( val_size ) ) :#information calculation
            temp = val_size[ i ] / att_size
            temp = temp * val_ent[ i ]
            info = info + temp
            i = i + 1
            
        gain = parent_entropy - info# gain calculated

        intrinsic_info = 0
        i = 0
        temp = 0
        while( i < len( val_size ) ) :#intinsic info calculation
            temp = val_size[ i ] / att_size 
            if( temp != 0 ) :
                temp = temp * ( mt.log( temp ) / mt.log( 2 ) ) 
            intrinsic_info = intrinsic_info + temp
            i = i + 1 
            
        if( intrinsic_info == 0 ):
            intrinsic_info = 1
        else:
            intrinsic_info = intrinsic_info * ( -1 )
    
        gr = ( gain / intrinsic_info )

        if( gr > gain_ratio or ff == 0 ) :
            gain_ratio = gr 
            attribute_selected = attribute
            rt_vals_attribute = vals_attribute
            initialized = 1
            if( ff == 0 ) :
                ff = 1 
#         x, y = vals_attribute.shape        
        vals_attribute = nm.empty( [ 0, 0 ] )
#     print("Returning selecetd attribute as ", attribute_selected, " with values ", rt_vals_attribute )
    if( initialized == 1 ) :
        return attribute_selected, rt_vals_attribute.tolist()
    else :
        return "NONE", "NONE"


    

In [29]:
data = ps.read_csv("Training.csv") #here data is DataFrame object
row, col = data.shape
print("Rows ", row, "Columns ", col )
col_names = list( data.columns.values )
lable = "left"

to_be_selected = []
i = 0

# while( i < col ) :#Running while loop for each column to filter out categoric attribute
#     if( col_names[ i ] != lable and len( data[ col_names[ i ] ].unique() ) <= 10 ):   
#         to_be_selected.append( col_names[ i ] )    
#     i = i + 1   
categorical_attribute = [ "Work_accident", "promotion_last_5years", "Department", "salary" ] 
numerical_attribute = [ "satisfaction_level", "last_evaluation", "number_project", "average_montly_hours", "time_spend_company" ]
to_be_selected = [ "Work_accident", "promotion_last_5years", "Department", "salary", "satisfaction_level", "last_evaluation", "number_project", "average_montly_hours", "time_spend_company" ]
Entropy_Def = calc_entropy( data, lable )
#entropy for first attribute
#Entropy of first attribute will be same irrespective of attribute we are selecting for it, 
#as data available for it is the whole dataset

ff = 0 #to make first gain ratio enter correctly
gain_ratio = 0
attribute_selected, attribute_values = select_attribute( data, to_be_selected, numerical_attribute, Entropy_Def )

print("Attribute selected for root ", attribute_selected )

if( attribute_selected not in numerical_attribute ) :
    to_be_selected.remove( attribute_selected )
#     print("Not a numerical attribute ")
numeric = 0
if attribute_selected in numerical_attribute :
    numeric = 1
root = Node( attribute_selected, data, attribute_values, to_be_selected, numeric, Entropy_Def ) #Creating root node
print( "Root : ", root.attribute_name, "\nValues ", attribute_values)#Updating here

nodes = []

nodes.append( root )

while( len( nodes ) > 0 ) :
    current = nodes.pop( 0 )
    print("\n\n New node ")
    attribute = current.attribute_name
#     print( "\n\nParent ", attribute )
    curr_data = current.data #Parent node's data
    select_from = copy.deepcopy( current.select_from )
    values = current.attribute_values
    x, y = curr_data.shape
    print("Parent ", attribute, ) 
    
    if( len( values ) == 1 ) :
        print("Only single value left ")
        continue
#     if( x <= 60 ):
#         print("Less than 60 rows left ")
#         continue
    
    curr_entropy = current.entropy
    if( curr_entropy != 0 and len( select_from ) > 0 ) :
        if( current.numeric == 1 ) :
                first = 0
                for val in values :
                    if( first == 0 ) :
                        node_data = curr_data[ curr_data[ attribute ] <= val ]
                    else :
                        node_data = curr_data[ curr_data[ attribute ] <= val ]
                        node_data = node_data[ node_data[ attribute ] > pre ]
                    
                    attribute_selected, attribute_values = select_attribute( node_data, select_from, numerical_attribute, curr_entropy )#current data, list of attributes from selection to be made,
                                                                                                                                        #categoric attributes with their all possible values        
                    pre = val#to maintain previous val, so that we canm check greater than previous and >= current val     
                    first = 1
                    
                    if( attribute_selected == "NONE" ) :
                        print("Dropping ", val )
                        current.attribute_values.remove( val )
                        print( "None of the attributes were selected ")
                        continue
                        
                    node_select_from = copy.deepcopy( select_from )
#                     node_select_from.remove( attribute_selected )
                    numeric = 0
                    if attribute_selected in numerical_attribute :
                        numeric = 1
                    node_entropy = calc_entropy( node_data, lable )
                    
                    if( numeric == 0 ) :
                        node_select_from.remove( attribute_selected )
                    elif( numeric == 1 ) :
                        if( len( node_data[ attribute_selected ].unique().tolist() ) == 1 ) :
                            node_select_from.remove( attribute_selected )
                            
                    child = Node( attribute_selected, node_data, attribute_values, node_select_from, numeric, node_entropy )
                    current.add_child( child )
                    if( node_entropy != 0 ):
                        nodes.append( child )
                    else :
                        print("Entropy zero for ", attribute_selected )    
                    print("child ", attribute_selected  )    
        else :
            for val in values :
                node_data = curr_data[ curr_data[ attribute ] == val ]#Child node's data
                attribute_selected, attribute_values = select_attribute( node_data, select_from, numerical_attribute, curr_entropy )#current data, list of attributes from selection to be made,
                                                                                           #categoric attributes with their all possible values
                if( attribute_selected == "NONE" ) :
                    print("Dropping ", val )
                    current.attribute_values.remove( val )
                    print( "None of the attributes were selected ")
                    continue
                        
                node_select_from = copy.deepcopy( select_from )
                
                numeric = 0
                if attribute_selected in numerical_attribute :
                    numeric = 1
                node_entropy = calc_entropy( node_data, lable )    
                
                if( numeric == 0 ) :
                        node_select_from.remove( attribute_selected )
                elif( numeric == 1 ) :
                    if( len( node_data[ attribute_selected ].unique().tolist() ) == 1 ) :
                        node_select_from.remove( attribute_selected )
                
                child = Node( attribute_selected, node_data, attribute_values, node_select_from, numeric, node_entropy )
                current.add_child( child )
            
                if( node_entropy != 0 ):
                    nodes.append( child )
                else :
                    print("Entropy zero for ", attribute_selected )
                print("child ", attribute_selected  )    
    #             print( "Child ", attribute_selected, "for value ", val )
#             if( len( node_select_from ) != 0 ) :
    if( len( current.childrens ) > 0 ) :
        current.data.drop( columns = current.data.columns )        


Rows  8993 Columns  10
Attribute selected for root  Work_accident
Root :  Work_accident 
Values  [0, 1]


 New node 
Parent  Work_accident
child  Department
child  promotion_last_5years


 New node 
Parent  Department
child  average_montly_hours
child  promotion_last_5years
child  salary
child  satisfaction_level
child  promotion_last_5years
child  satisfaction_level
child  promotion_last_5years
child  average_montly_hours
child  salary
child  satisfaction_level


 New node 
Parent  promotion_last_5years
child  satisfaction_level
child  satisfaction_level


 New node 
Parent  average_montly_hours
child  last_evaluation
child  last_evaluation
child  salary
child  average_montly_hours
child  satisfaction_level


 New node 
Parent  promotion_last_5years
Entropy zero for  last_evaluation
child  last_evaluation
child  satisfaction_level


 New node 
Parent  salary
child  satisfaction_level
child  promotion_last_5years
child  average_montly_hours


 New node 
Parent  satisfaction_level
child

Entropy zero for  promotion_last_5years
child  promotion_last_5years
child  time_spend_company
child  time_spend_company


 New node 
Parent  promotion_last_5years
child  last_evaluation
Entropy zero for  average_montly_hours
child  average_montly_hours


 New node 
Parent  average_montly_hours
child  number_project
child  satisfaction_level
child  number_project
child  last_evaluation
child  last_evaluation


 New node 
Parent  average_montly_hours
child  satisfaction_level
child  number_project
child  last_evaluation
Entropy zero for  number_project
child  number_project
child  salary


 New node 
Parent  time_spend_company
child  average_montly_hours
Entropy zero for  number_project
child  number_project
Entropy zero for  salary
child  salary
Entropy zero for  number_project
child  number_project
Entropy zero for  time_spend_company
child  time_spend_company


 New node 
Parent  average_montly_hours
Entropy zero for  time_spend_company
child  time_spend_company
Entropy zero for  tim

child  number_project
Entropy zero for  satisfaction_level
child  satisfaction_level
Entropy zero for  number_project
child  number_project
child  satisfaction_level
Entropy zero for  satisfaction_level
child  satisfaction_level


 New node 
Parent  time_spend_company
Dropping  3
None of the attributes were selected 
child  time_spend_company
Dropping  6
None of the attributes were selected 


 New node 
Parent  promotion_last_5years
child  last_evaluation
Dropping  1
None of the attributes were selected 


 New node 
Parent  satisfaction_level
child  number_project
child  number_project
Entropy zero for  salary
child  salary


 New node 
Parent  promotion_last_5years
child  average_montly_hours
Dropping  1
None of the attributes were selected 


 New node 
Parent  last_evaluation
child  number_project
child  number_project
child  number_project
child  average_montly_hours
child  number_project


 New node 
Parent  number_project
Dropping  2
None of the attributes were selected 
Entrop

 New node 
Parent  satisfaction_level
child  satisfaction_level
child  number_project
child  time_spend_company
child  time_spend_company
child  average_montly_hours


 New node 
Parent  satisfaction_level
child  satisfaction_level
Entropy zero for  satisfaction_level
child  satisfaction_level
Entropy zero for  last_evaluation
child  last_evaluation
Entropy zero for  satisfaction_level
child  satisfaction_level
Dropping  0.36
None of the attributes were selected 


 New node 
Parent  last_evaluation
Entropy zero for  time_spend_company
child  time_spend_company
child  satisfaction_level
Entropy zero for  last_evaluation
child  last_evaluation
Entropy zero for  satisfaction_level
child  satisfaction_level
Entropy zero for  number_project
child  number_project


 New node 
Parent  last_evaluation
Entropy zero for  number_project
child  number_project
Entropy zero for  time_spend_company
child  time_spend_company
Entropy zero for  time_spend_company
child  time_spend_company
Entropy zero 

child  satisfaction_level
Entropy zero for  number_project
child  number_project
Dropping  5
None of the attributes were selected 


 New node 
Parent  time_spend_company
Entropy zero for  number_project
child  number_project
Entropy zero for  satisfaction_level
child  satisfaction_level


 New node 
Parent  last_evaluation
Entropy zero for  satisfaction_level
child  satisfaction_level
Dropping  0.66
None of the attributes were selected 
Entropy zero for  satisfaction_level
child  satisfaction_level
Dropping  0.99
None of the attributes were selected 


 New node 
Parent  satisfaction_level
Entropy zero for  average_montly_hours
child  average_montly_hours
Dropping  0.89
None of the attributes were selected 


 New node 
Parent  salary
Entropy zero for  satisfaction_level
child  satisfaction_level
Dropping  medium
None of the attributes were selected 


 New node 
Parent  satisfaction_level
Entropy zero for  time_spend_company
child  time_spend_company
Entropy zero for  time_spend_comp

...

Entropy zero for  number_project
child  number_project
Entropy zero for  last_evaluation
child  last_evaluation
child  average_montly_hours
Entropy zero for  number_project
child  number_project
Entropy zero for  number_project
child  number_project


 New node 
Parent  number_project
Entropy zero for  time_spend_company
child  time_spend_company
Entropy zero for  time_spend_company
child  time_spend_company
Entropy zero for  average_montly_hours
child  average_montly_hours
Entropy zero for  time_spend_company
child  time_spend_company


 New node 
Parent  last_evaluation
Entropy zero for  time_spend_company
child  time_spend_company
Entropy zero for  number_project
child  number_project
Entropy zero for  number_project
child  number_project
Entropy zero for  average_montly_hours
child  average_montly_hours
child  time_spend_company


 New node 
Parent  satisfaction_level
Dropping  0.16
None of the attributes were selected 
child  satisfaction_level


 New node 
Parent  average_montly_

child  number_project
Dropping  5
None of the attributes were selected 


 New node 
Parent  number_project
Entropy zero for  satisfaction_level
child  satisfaction_level
Entropy zero for  satisfaction_level
child  satisfaction_level
Dropping  6
None of the attributes were selected 


 New node 
Parent  number_project
Entropy zero for  satisfaction_level
child  satisfaction_level
Dropping  3
None of the attributes were selected 


 New node 
Parent  satisfaction_level
Entropy zero for  satisfaction_level
child  satisfaction_level
Dropping  0.31
None of the attributes were selected 
Entropy zero for  satisfaction_level
child  satisfaction_level
child  satisfaction_level


 New node 
Parent  time_spend_company
Dropping  2
None of the attributes were selected 
child  time_spend_company


 New node 
Parent  satisfaction_level
Dropping  0.8
None of the attributes were selected 


 New node 
Parent  number_project
Dropping  2
None of the attributes were selected 
Entropy zero for  number_pro

child  satisfaction_level
Dropping  269
None of the attributes were selected 


 New node 
Parent  average_montly_hours
child  number_project
child  satisfaction_level
Entropy zero for  satisfaction_level
child  satisfaction_level
child  time_spend_company
Dropping  287
None of the attributes were selected 


 New node 
Parent  time_spend_company
Entropy zero for  satisfaction_level
child  satisfaction_level
Entropy zero for  number_project
child  number_project
Dropping  6
None of the attributes were selected 


 New node 
Parent  last_evaluation
Entropy zero for  satisfaction_level
child  satisfaction_level
Dropping  0.61
None of the attributes were selected 
Entropy zero for  satisfaction_level
child  satisfaction_level
child  satisfaction_level


 New node 
Parent  time_spend_company
Entropy zero for  satisfaction_level
child  satisfaction_level
Entropy zero for  number_project
child  number_project
Dropping  5
None of the attributes were selected 


 New node 
Parent  last_evaluat

Entropy zero for  satisfaction_level
child  satisfaction_level
Dropping  6
None of the attributes were selected 


 New node 
Parent  average_montly_hours
Entropy zero for  satisfaction_level
child  satisfaction_level
Dropping  152
None of the attributes were selected 


 New node 
Parent  time_spend_company
Dropping  2
None of the attributes were selected 
Entropy zero for  time_spend_company
child  time_spend_company
Dropping  5
None of the attributes were selected 


 New node 
Parent  satisfaction_level
Dropping  0.33
None of the attributes were selected 


 New node 
Parent  time_spend_company
Dropping  3
None of the attributes were selected 
Entropy zero for  time_spend_company
child  time_spend_company


 New node 
Parent  satisfaction_level
Dropping  0.5
None of the attributes were selected 
child  satisfaction_level


 New node 
Parent  satisfaction_level
Dropping  0.83
None of the attributes were selected 


 New node 
Parent  number_project
Entropy zero for  satisfaction_lev

Dropping  0.57
None of the attributes were selected 


 New node 
Parent  satisfaction_level
Dropping  0.55
None of the attributes were selected 


 New node 
Parent  satisfaction_level
Dropping  0.38
None of the attributes were selected 


 New node 
Parent  number_project
Dropping  4
None of the attributes were selected 


 New node 
Parent  satisfaction_level
Dropping  0.4
None of the attributes were selected 


 New node 
Parent  average_montly_hours
child  last_evaluation
Dropping  142
None of the attributes were selected 
Entropy zero for  satisfaction_level
child  satisfaction_level
Entropy zero for  satisfaction_level
child  satisfaction_level


 New node 
Parent  last_evaluation
Dropping  0.41
None of the attributes were selected 
Entropy zero for  last_evaluation
child  last_evaluation


 New node 
Parent  satisfaction_level
Dropping  0.91
None of the attributes were selected 


 New node 
Parent  satisfaction_level
Dropping  0.74
None of the attributes were selected 


 New 

In [32]:
def no_val( c_node, row_i ) :
    count_rows = 0
    count_ones = 0
    child = 1
    val_no = 0
    for val in c_node.childrens :
        next_node = val
        while( child == 1 ) : #Traversing whole tree
#             print( "Node's attribute ", next_node.attribute_name, ", with values ", next_node.attribute_values )
            val_row_i = row_i[ next_node.attribute_name ] #Value in row red for attribute, the node is of( attribute on which node is defined )
#             print("Value in input row ", val_row_i )

            if( len( next_node.childrens ) > 0 ):
                if val_row_i in next_node.attribute_values :
                    ind = next_node.attribute_values.index( val_row_i )
                    next_node = next_node.childrens[ ind ]
                else :
                    rt = no_val( next_node, row_i )
                    val_no = 1
                    child = 0
            else :
                child = 0
         
        n_row, n_col = next_node.data.shape
    
        if( val_no == 1 ) :
            if( rt == 1 ) :
                count_ones = count_ones + 1
            count_rows = count_rows + 1
        elif( n_row == 1 ) : #Calculating Prediction
            res_row = next_node.data.iloc[ 0 ]
            if( res_row[ lable ] == 1 ) :
                count_ones = count_ones + 1
            count_rows = count_rows + 1
        else:
            j = 0
            count_1 = 0
            count_t = 0
            while( j < n_row ) :
                resm_row = next_node.data.iloc[ j ]
                if( resm_row[ lable ] == 1 ) :
                    count_1 = count_1 + 1
                count_t = count_t + 1
                j = j + 1

            if( count_1 / count_t  >= 0.5 ) :
                count_ones = count_ones + 1
            count_rows = count_rows + 1
            
    if( count_ones / count_rows >= 0.5 ) :
        rt = 1
    else :
        rt = 0
        
    return rt


validation_data = ps.read_csv("Valid.csv") #here data is DataFrame object
v_row, v_col = validation_data.shape
print("Rows :", v_row, ", Columns : ", v_col)

i = 0
A_yes_P_no = 0
A_yes_P_yes = 0
A_no_P_no = 0
A_no_P_yes = 0

while( i < v_row ) :
    row_i = validation_data.iloc[ i ]
    child = 1
    next_node = root 
#     print( "\n\n New test row ", i + 1 )
    val_no = 0
    while( child == 1 ) : #Traversing whole tree
#         print( "Node's attribute ", next_node.attribute_name, ", with values ", next_node.attribute_values )
        val_row_i = row_i[ next_node.attribute_name ] #Value in row red for attribute, the node is of( attribute on which node is defined )
#         print("Value in input row ", val_row_i )
                
        if( len( next_node.childrens ) > 0 ):#Checking if the node has childrens or not
            if val_row_i in next_node.attribute_values :#Checking if the value of row is in node or not( it can be null as well in that case it won't be in node )
                #If row value is not in the node, then we go to all childrens and check for next attribute( node ) and recursively check for it
                ind = next_node.attribute_values.index( val_row_i )
                next_node = next_node.childrens[ ind ]
            else :
                rt = no_val( next_node, row_i )
                val_no = 1
                child = 0
        else :
            child = 0
    
    n_row, n_col = next_node.data.shape
    
    if( val_no == 1 ) :
        if( rt == 1 ) :
            if( row_i[ lable ] == 1 ) :
                A_yes_P_yes = A_yes_P_yes + 1
            else :
                A_no_P_yes = A_no_P_yes + 1
        else:
            if( row_i[ lable ] == 0 ) :
                A_no_P_no = A_no_P_no + 1
            else :
                A_yes_P_no = A_yes_P_no + 1
    
    elif( n_row == 1 ) : #Calculating Prediction
        res_row = next_node.data.iloc[ 0 ]
        if( res_row[ lable ] == 1 ) :
            if( row_i[ lable ] == 1 ) :
                A_yes_P_yes = A_yes_P_yes + 1
            else :
                A_no_P_yes = A_no_P_yes + 1
        else:
            if( row_i[ lable ] == 0 ) :
                A_no_P_no = A_no_P_no + 1
            else :
                A_yes_P_no = A_yes_P_no + 1
                    
    else:
        j = 0
        count_1 = 0
        count_t = 0
        while( j < n_row ) :
            resm_row = next_node.data.iloc[ j ]
            if( resm_row[ lable ] == 1 ) :
                count_1 = count_1 + 1
            count_t = count_t + 1
            j = j + 1
       
        if( count_1 / count_t  >= 0.5 ) :
            if( row_i[ lable ] == 1 ) :
                A_yes_P_yes = A_yes_P_yes + 1
            else :
                A_no_P_yes = A_no_P_yes + 1
        else :
            if( row_i[ lable ] == 0 ) :
                A_no_P_no = A_no_P_no + 1
            else :
                A_yes_P_no = A_yes_P_no + 1
        
        
    i =  i + 1
    
print("Number of rows red : ", i )

print(" Actual yes Predicted yes ", A_yes_P_yes )    
print(" Actual No Predicted yes ", A_no_P_yes )
print(" Actual yes Predicted no ", A_yes_P_no )
print(" Actual_no_Predicted_no", A_no_P_no )
# print( data )

Rows : 2245 , Columns :  10


IndexError: list index out of range